In [19]:
#Import all the nessacary libraries to run the pose estimator

#Import mediapipe to be used for the model
import mediapipe as mp
#Import opencv for rendaring and drawing capabilities
import cv2

import numpy as np #Handle numpy arrays
import pandas as pd #Handle tabular data
import os #Handle folder structure
import pickle #Save and oad ML model

In [22]:
holistic_model = mp.solutions.holistic 

In [9]:
#Import the model from the binary file
with open('rfc_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [11]:
#Calculate angle between 3 landmark points
def calculate_pose_angle(start_point, mid_point, end_point):
    #Convert landmark coords to numpy array
    start_point = np.array(start_point) 
    mid_point = np.array(mid_point) 
    end_point = np.array(end_point) 
    
    max_angle = 180.0
    
    #[0] = x, [1] = y, [2] = z
    radians = np.arctan2(end_point[1] - mid_point[1], end_point[0] - mid_point[0]) - np.arctan2(start_point[1] - mid_point[1], start_point[0] - mid_point[0])
    #Convert to an angle
    angle = np.abs(radians * max_angle / np.pi)
    
    if angle > max_angle:
        angle = 360 - angle
        
    return angle 

In [26]:
#PREDICT AND DISPLAY THE RESULTS OF THE MODEL BY PASSING THE TEST VIDEO

#Connect the test video from the device
sample_video = cv2.VideoCapture('datasets/invalid/5.invalid.mp4')


# Curl counter variables
reps = 0 
command = None

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        status, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if status == False:
            print("Exit!")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)        
        #Predict the coordinates of the landmarks (results screen)
        try:
            #Extracting all the landmarks of the pose as an array
            pose_landmarks_array = result_frame.pose_landmarks.landmark
            #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
            pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] 
                                                    for landmark in pose_landmarks_array]).flatten() 
                                          if result_frame.pose_landmarks else np.zeros(33*4))

            #Pass the numpy array into a data frame
            features = pd.DataFrame([pose_landmarks_nparray])
            #Store the top class of the prediction
            pose_class_status = model.predict(features.values)[0]
            #Store the probability of the prediction
            pose_class_status_prob = model.predict_proba(features.values)[0]
            #Retrieve x and y coords of the landmark points
            left_shoulder = [pose_landmarks_array[holistic_model.PoseLandmark.LEFT_SHOULDER.value].x, 
                             pose_landmarks_array[holistic_model.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [pose_landmarks_array[holistic_model.PoseLandmark.LEFT_ELBOW.value].x, 
                          pose_landmarks_array[holistic_model.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [pose_landmarks_array[holistic_model.PoseLandmark.LEFT_WRIST.value].x, 
                          pose_landmarks_array[holistic_model.PoseLandmark.LEFT_WRIST.value].y]
            right_shoulder = [pose_landmarks_array[holistic_model.PoseLandmark.RIGHT_SHOULDER.value].x, 
                              pose_landmarks_array[holistic_model.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [pose_landmarks_array[holistic_model.PoseLandmark.RIGHT_ELBOW.value].x, 
                           pose_landmarks_array[holistic_model.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [pose_landmarks_array[holistic_model.PoseLandmark.RIGHT_WRIST.value].x, 
                           pose_landmarks_array[holistic_model.PoseLandmark.RIGHT_WRIST.value].y]
            left_angle = calculate_pose_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = calculate_pose_angle(right_shoulder, right_elbow, right_wrist)
            
            # Curl counter logic
            if left_angle > 140 and right_angle > 140:
                command = "Down"
            if left_angle < 90 and right_angle > 140 and command =='Down':
                command = "Up"
                if pose_class_status.split(' ')[0].lower == "correct":
                    reps += 1
                    print(reps)
                    
            #Set a rectangle box to display the results of the prediction in the video frame
            #rectangle(container, top_coord, bottom_coord, color, line_thickness)
            cv2.rectangle(bgr_frame, (0,0), (250, 150), (245, 117, 16), -1)
            
            cv2.putText(bgr_frame, 'Class'
                        , (105,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and display the top class of the prediction
            cv2.putText(bgr_frame, pose_class_status.split(' ')[0]
                        , (100,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(bgr_frame, 'Probability'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and display the maximum probability
            cv2.putText(bgr_frame, str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(bgr_frame, 'Reps'
                        , (105,62), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Display the no of repatitions
            cv2.putText(bgr_frame, str(reps)
                        , (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(bgr_frame, 'Stage'
                        , (15,62), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Display the command
            cv2.putText(bgr_frame, command
                        , (10,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
 
        except:
            pass
                        
        #Display the frames  

        cv2.imshow('Results Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    print("Class:", pose_class_status, " Probability:", str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2)))


sample_video.release()
cv2.destroyAllWindows()

Exit!
Class: Incorrect  Probability: 1.0
